In [1]:
!pip install datasets numpy librosa soundfile

!rm -rf mozilla-common-*

In [2]:
downloadDataset = True

if (downloadDataset):
    from datasets import load_dataset
    cv_13 = load_dataset("mozilla-foundation/common_voice_13_0", "it", split="validation")
    cv_13.save_to_disk("mozilla-common-voices")

Saving the dataset (0/2 shards):   0%|          | 0/15086 [00:00<?, ? examples/s]

In [3]:
from datasets import load_from_disk
import numpy as np

arrow_datasets_reloaded = load_from_disk("mozilla-common-voices")

In [4]:
pip install "soundfile>=0.12.1"

Note: you may need to restart the kernel to use updated packages.


In [5]:
arrow_datasets_reloaded[0]

{'client_id': 'f272af5faf3641289daebbe716f746e188391bb6a6d30bd553023e1230ff692305143f7766d40fed6da67e9915a31163ea5412a99769f0309b175433b476e622',
 'path': '/home/jovyan/.cache/huggingface/datasets/downloads/extracted/cf092477572937be3dffa6da0cc8bea1412b82f2ba3acb886c92a5e60a9486f9/it_dev_0/common_voice_it_19348529.mp3',
 'audio': {'path': 'common_voice_it_19348529.mp3',
  'array': array([0.00000000e+00, 2.61833605e-14, 2.73162383e-14, ...,
         1.29989053e-06, 1.64870050e-06, 1.44956334e-06]),
  'sampling_rate': 48000},
 'sentence': 'Nella camera da letto.',
 'up_votes': 2,
 'down_votes': 1,
 'age': '',
 'gender': '',
 'accent': '',
 'locale': 'it',
 'segment': '',
 'variant': ''}

In [6]:
from IPython.display import Audio
# Play the audio
Audio(data=arrow_datasets_reloaded[0]['audio']['array'], rate=arrow_datasets_reloaded[0]['audio']['sampling_rate'])

In [7]:
audio = arrow_datasets_reloaded[0]['audio']['array']
sampling_rate = arrow_datasets_reloaded[0]['audio']['sampling_rate']

In [8]:
# https://americodias.com/docs/python/audio_python.md
def fir_band_pass(samples, fs, fL, fH, NL, NH):
    # Referece: https://fiiir.com

    fH = fH / fs
    fL = fL / fs

    # Compute a low-pass filter with cutoff frequency fH.
    hlpf = np.sinc(2 * fH * (np.arange(NH) - (NH - 1) / 2.))
    hlpf *= np.blackman(NH)
    hlpf /= np.sum(hlpf)
    # Compute a high-pass filter with cutoff frequency fL.
    hhpf = np.sinc(2 * fL * (np.arange(NL) - (NL - 1) / 2.))
    hhpf *= np.blackman(NL)
    hhpf /= np.sum(hhpf)
    hhpf = -hhpf
    hhpf[int((NL - 1) / 2)] += 1
    # Convolve both filters.
    h = np.convolve(hlpf, hhpf)
    # Applying the filter to a signal s can be as simple as writing
    s = np.convolve(samples, h)

    return s

# Using custom SSB filter
samples_filtered = fir_band_pass(audio, sampling_rate, 300, 2700, 461, 461)

In [9]:
from IPython.display import Audio
# Play the audio
Audio(data=samples_filtered, rate=sampling_rate)

In [ ]:
updated_dataset = arrow_datasets_reloaded.map(lambda example: {"audio_filtered": fir_band_pass(example['audio']['array'], example['audio']['sampling_rate'], 300, 2700, 461, 461)}, writer_batch_size=1000)

Map:   0%|          | 0/15086 [00:00<?, ? examples/s]

In [ ]:
updated_dataset.save_to_disk("mozilla-common-voices-filtered")